In [253]:
from astropy.io import fits
import numpy as np
from numpy import fft
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from astropy import units as u
from spectral_cube import SpectralCube

In [216]:
hdul = fits.open(".data/other_data/ngc2403.fits")

In [259]:
# Extract a 3D point cloud from a FITS file containing hyperspectral data.
# 
# Parameters:
# - fits_path (str): Path to the FITS file.
# - intensity_threshold (float): Minimum intensity value to consider a point.
# - spatial_scale (float): Conversion factor from pixels to physical spatial units.
# - velocity_scale (float): Conversion factor from channels to velocity units.
# - velocity_offset (float): Velocity value corresponding to channel 0.
# 
# Returns:
# - np.ndarray: Nx3 array representing the 3D point cloud (x, y, v).

fits_path = ".data/other_data/ngc2403.fits"
intensity_threshold=0.001 
spatial_scale=0.5, 
velocity_scale=1.0, 
velocity_offset=0.0

# Load data from FITS file
hdul = fits.open(fits_path)
data_cube = hdul[0].data.copy()
header = hdul[0].header.copy()


# Assuming data_cube is (velocity, y, x)
indices = np.argwhere(data_cube > intensity_threshold)

# Extract positions and velocities
x_coords = indices[:, 2] * spatial_scale
y_coords = indices[:, 1] * spatial_scale
velocities = indices[:, 0] * velocity_scale + velocity_offset
intensities = data_cube[indices[:, 0], indices[:, 1], indices[:, 2]]

# Combine into point cloud
pc = np.column_stack((x_coords, y_coords, velocities, intensities))

In [ ]:
# velocity_kms = hdul[0].data[3]
# velocity_ms = (velocity_kms * u.km / u.s).to(u.m / u.s).value  
# hdul[0].data[3] = velocity_ms
# 
# hdul[0].header['CUNIT3'] = "m/s"
# 
# hdul.writeto('.data/other_data/ngc2403_fixed.fits', overwrite=True)

In [265]:
cube = SpectralCube.read(".data/other_data/ngc2403_fixed.fits")

In [ ]:
hypercube.

AttributeError: ("Header's '%s' card is missing, but required for this operation.", 'CUNIT3CDELT3')

In [219]:
df = pd.DataFrame(pc)
df.columns = ["x", "y", "z", "intensity"]

In [220]:
df

,x,y,z,intensity
0,62.5,0.5,0.0,0.001111
1,62.0,1.0,0.0,0.001177
2,62.5,1.0,0.0,0.001201
3,62.0,1.5,0.0,0.001197
4,62.5,1.5,0.0,0.001057
...,...,...,...,...
193490,56.5,73.0,61.0,0.001075
193491,56.0,73.5,61.0,0.001278
193492,56.5,73.5,61.0,0.001376
193493,56.0,74.0,61.0,0.001174


In [221]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(df.iloc[:,:3].values.tolist())

In [222]:
intensity = df['intensity'].values*100
volume_normalized = (intensity - intensity.min()) / (intensity.max() - intensity.min())
colors = plt.cm.plasma(intensity)[:, :3]  # Use a colormap (e.g., viridis)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [223]:
o3d.visualization.draw_geometries([pcd])

MESA: error: ZINK: failed to choose pdev
glx: failed to create drisw screen
